In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
client = bigquery.Client()

In [9]:
dataset_ref = client.dataset("openaq", project = "bigquery-public-data")

In [10]:
dataset = client.get_dataset(dataset_ref)

In [13]:
tables = list(client.list_tables(dataset))

In [17]:
for table in tables:
    print (table.table_id)

global_air_quality


In [18]:
table_ref = dataset_ref.table("global_air_quality")
table = client.get_table(table_ref)

In [20]:
table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

In [23]:
client.list_rows(table, max_results = 5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


### Pollutants that are measured in Bosnia and Herzegovina

In [57]:
query_job = client.query("""
    SELECT DISTINCT pollutant
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'BA'
    """)

In [58]:
results = query_job.result()

In [59]:
results.to_dataframe()

,pollutant
0,no2
1,pm10
2,co
3,o3
4,so2
5,pm25


### I assumed that all those location are divided in some regions

In [68]:
query_job = client.query("""
    SELECT DISTINCT source_name
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = "BA" 
    """)

In [69]:
results = query_job.result()

In [70]:
results.to_dataframe()

,source_name
0,Bosnia
1,Bosnia2
2,Bosnia_Tuzlanski


### Let's see all those locations for every region

In [93]:
query_job = client.query("""
    SELECT DISTINCT location
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = "BA" AND source_name = "Bosnia_Tuzlanski"
    """)

In [94]:
results = query_job.result()

In [95]:
tuzla = results.to_dataframe()

In [98]:
tuzla.columns = ["Bosnia_Tuzlanski"]
tuzla

,Bosnia_Tuzlanski
0,Živinice
1,Bukinje
2,BKC
3,Lukavac
4,Skver


In [99]:
query_job = client.query("""
    SELECT DISTINCT location
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = "BA" AND source_name = "Bosnia"
    """)

In [100]:
results = query_job.result()

In [101]:
result = results.to_dataframe()

In [112]:
result.columns = ["Bosnia"]
result

,Bosnia
0,Rasadnik
1,IvanSedlo
2,Ilidža
3,Otoka
4,Vijećnica
5,Bjelave
6,Ilijaš
7,Doboj
8,Radakovo
9,Brist


In [107]:
query_job = client.query("""
    SELECT DISTINCT location
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = "BA" AND source_name = "Bosnia2"
    """)

In [108]:
results = query_job.result()

In [109]:
result = results.to_dataframe()

In [113]:
result.columns = ["Bosnia2"]
result

,Bosnia2
0,Rasadnik
1,IvanSedlo
2,Ilidža
3,Otoka
4,Vijećnica
5,Bjelave
6,Ilijaš
7,Doboj
8,Radakovo
9,Brist


##### Bosnia2 contains the same locations as Bosnia except that Bosnia has mobilna(ilidza). So, I am going to analyze all those locations from Bosnia2 and just one from Bosnia (mobilna Ilidza).

### Select all measurments from this day

In [158]:
query_job = client.query("""
    SELECT *
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = "BA" AND CAST(timestamp as date) = CAST("2020-01-15" as date)
    LIMIT 10
""")

In [159]:
results = query_job.result()

In [160]:
results.to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,US Diplomatic Post: Sarajevo,Sarajevo,BA,pm25,295.0,2020-01-15 19:00:00+00:00,µg/m³,Sarajevo,43.856667,18.398205,1.0
1,Živinice,Živinice,BA,co,5352.0,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.453888,18.648333,1.0
2,Živinice,Živinice,BA,no2,42.2,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.453888,18.648333,1.0
3,Živinice,Živinice,BA,so2,277.2,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.453888,18.648333,1.0
4,Bukinje,Tuzla,BA,co,1716.0,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.523613,18.600277,1.0
5,Bukinje,Tuzla,BA,o3,7.9,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.523613,18.600277,1.0
6,Bukinje,Tuzla,BA,pm25,60.0,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.523613,18.600277,1.0
7,Bukinje,Tuzla,BA,so2,130.2,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.523613,18.600277,1.0
8,BKC,Tuzla,BA,co,6557.0,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.531944,18.654722,1.0
9,BKC,Tuzla,BA,no2,40.3,2020-01-15 19:00:00+00:00,µg/m³,Bosnia_Tuzlanski,44.531944,18.654722,1.0


In [157]:
results.total_rows

10